# Linear Regression using Pyspark

In [0]:
#create sparksession object
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('lin_reg').getOrCreate()

In [0]:
#import Linear Regression from spark's MLlib
from pyspark.ml.regression import LinearRegression

In [0]:
#Load the dataset
df=spark.read.csv('dbfs:/FileStore/tables/Linear_regression_dataset.csv',inferSchema=True,header=True)

In [0]:
 #validate the size of data
print((df.count(), len(df.columns)))

(1232, 6)


In [0]:
#explore the data
df.printSchema()

root
 |-- var_1: integer (nullable = true)
 |-- var_2: integer (nullable = true)
 |-- var_3: integer (nullable = true)
 |-- var_4: double (nullable = true)
 |-- var_5: double (nullable = true)
 |-- output: double (nullable = true)



In [0]:
#view statistical measures of data 
df.describe().show(5,False)

+-------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|var_1            |var_2            |var_3             |var_4               |var_5               |output             |
+-------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+
|count  |1232             |1232             |1232              |1232                |1232                |1232               |
|mean   |715.0819805194806|715.0819805194806|80.90422077922078 |0.3263311688311693  |0.25927272727272715 |0.39734172077922014|
|stddev |91.5342940441652 |93.07993263118064|11.458139049993724|0.015012772334166148|0.012907228928000298|0.03326689862173776|
|min    |463              |472              |40                |0.277               |0.214               |0.301              |
|max    |1009             |1103             |116               |0.373               |0.294               |0.491

In [0]:
#sneak into the dataset
df.head(3)

Out[7]: [Row(var_1=734, var_2=688, var_3=81, var_4=0.328, var_5=0.259, output=0.418),
 Row(var_1=700, var_2=600, var_3=94, var_4=0.32, var_5=0.247, output=0.389),
 Row(var_1=712, var_2=705, var_3=93, var_4=0.311, var_5=0.247, output=0.417)]

In [0]:
#import corr function from pyspark functions
from pyspark.sql.functions import corr

In [0]:
# check for correlation
df.select(corr('var_1','output')).show()

+-------------------+
|corr(var_1, output)|
+-------------------+
| 0.9187399607627283|
+-------------------+



In [0]:
#import vectorassembler to create dense vectors
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [0]:
#select the columns to create input vector
df.columns

Out[11]: ['var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'output']

In [0]:
#create the vector assembler 
vec_assmebler=VectorAssembler(inputCols=['var_1', 'var_2', 'var_3', 'var_4', 'var_5'],outputCol='features')

In [0]:
#transform the values
features_df=vec_assmebler.transform(df)

In [0]:
#validate the presence of dense vectors 
features_df.printSchema()

root
 |-- var_1: integer (nullable = true)
 |-- var_2: integer (nullable = true)
 |-- var_3: integer (nullable = true)
 |-- var_4: double (nullable = true)
 |-- var_5: double (nullable = true)
 |-- output: double (nullable = true)
 |-- features: vector (nullable = true)



In [0]:
#view the details of dense vector
features_df.select('features').show(5,False)

+------------------------------+
|features                      |
+------------------------------+
|[734.0,688.0,81.0,0.328,0.259]|
|[700.0,600.0,94.0,0.32,0.247] |
|[712.0,705.0,93.0,0.311,0.247]|
|[734.0,806.0,69.0,0.315,0.26] |
|[613.0,759.0,61.0,0.302,0.24] |
+------------------------------+
only showing top 5 rows



In [0]:
#create data containing input features and output column
model_df=features_df.select('features','output')

In [0]:
model_df.show(5,False)

+------------------------------+------+
|features                      |output|
+------------------------------+------+
|[734.0,688.0,81.0,0.328,0.259]|0.418 |
|[700.0,600.0,94.0,0.32,0.247] |0.389 |
|[712.0,705.0,93.0,0.311,0.247]|0.417 |
|[734.0,806.0,69.0,0.315,0.26] |0.415 |
|[613.0,759.0,61.0,0.302,0.24] |0.378 |
+------------------------------+------+
only showing top 5 rows



In [0]:
#size of model df
print((model_df.count(), len(model_df.columns)))

(1232, 2)


### Split Data - Train & Test sets


In [0]:
#split the data into 70/30 ratio for train test purpose
train_df,test_df=model_df.randomSplit([0.7,0.3])

In [0]:
print((train_df.count(), len(train_df.columns)))

(861, 2)


In [0]:
print((test_df.count(), len(test_df.columns)))

(371, 2)


In [0]:
train_df.describe().show()

+-------+-------------------+
|summary|             output|
+-------+-------------------+
|  count|                861|
|   mean| 0.3972624854819972|
| stddev|0.03310635922684736|
|    min|              0.311|
|    max|              0.484|
+-------+-------------------+



## Build Linear Regression Model 

In [0]:
#Build Linear Regression model 
lin_Reg=LinearRegression(labelCol='output')

In [0]:
#fit the linear regression model on training data set 
lr_model=lin_Reg.fit(train_df)

In [0]:
lr_model.intercept

Out[25]: 0.18718020152749912

In [0]:
print(lr_model.coefficients)

[0.0003320295737682125,5.900842906053222e-05,0.000249704600864646,-0.6660064252246255,0.491902401218831]


In [0]:
training_predictions=lr_model.evaluate(train_df)

In [0]:
training_predictions.meanSquaredError

Out[28]: 0.00014710156200132394

In [0]:
training_predictions.r2

Out[29]: 0.8656309835461388

In [0]:
#make predictions on test data 
test_results=lr_model.evaluate(test_df)

In [0]:
#view the residual errors based on predictions 
test_results.residuals.show(10)

+--------------------+
|           residuals|
+--------------------+
|-0.01422653194133...|
|-0.01290645633993...|
|-0.01484927140022...|
| 0.00903946476818579|
|-0.01166274315109...|
|0.006775806296774822|
|-0.01155492387796...|
|-0.01000024846540...|
|-0.00854736408952...|
|-0.00180467656642...|
+--------------------+
only showing top 10 rows



In [0]:
#coefficient of determination value for model
test_results.r2

Out[32]: 0.8767740269312815

In [0]:
test_results.rootMeanSquaredError

Out[33]: 0.011807212288962738

In [0]:
test_results.meanSquaredError

Out[34]: 0.0001394102620366327